In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
from matplotlib.backends.backend_pdf import PdfPages
import openai
#OpenAI API key
openai.api_key = ''
# Read the data
KamickData = pd.read_csv(r"C:\Users\daddy\Downloads\Kamick Lead Times Most recent_Re - in.csv")
PLSLeadTimes = pd.read_csv(r"C:\Users\daddy\Downloads\PLS Project Order History Jan 1 20 - Jun 30 23.xlsm - Jan 1 20 - Jun 30 23 (1).csv")
# Extract SIN numbers from 'Customer Part Number' column in PLSLeadTimes
sin_numbers_pls = set(PLSLeadTimes['Customer Part Number'])
# Filter 'Name' column in KamickData for SIN numbers in PLSLeadTimes
result = KamickData[KamickData['Name'].isin(sin_numbers_pls)]
# Replace 'Description' values not contained within the sin_numbers_pls with an empty string
result.loc[~result['Name'].isin(sin_numbers_pls), 'Description'] = ''
# Merge the resulting data
result = result.merge(PLSLeadTimes[['Customer Part Number', 'Ave Lead Time']], left_on='Name', right_on='Customer Part Number', how='left')
# Rename columns 'Purchase Lead Time' to 'Kamick Lead Time' and 'Ave Lead Time' to 'PLS Lead Time' for professionalism
result.rename(columns={'Purchase Lead Time': 'Kamick Lead Time', 'Ave Lead Time': 'PLS Lead Time'}, inplace=True)
# Reset the index of the result DataFrame because we're not insane
result.reset_index(drop=True, inplace=True)
result['Kamick_PLS_LeadTimeDifference'] = result['Kamick Lead Time'] - result['PLS Lead Time']
# Define include and exclude words so we can differentiate wires and cables from actual mechanical parts to avoid skewing the TRUE variance of the data
exclude_words = ['wire', 'cable', 'conductor']
include_words = ['wire', 'cable', 'conductor']
# Filter dataframes and remove duplicates as well as calculating the Lead time differences between Kamick and PLS
Result = result[~result['Description'].fillna('').str.contains('|'.join(exclude_words), case=False)].drop_duplicates()
result_WIRE = result[result['Description'].fillna('').str.contains('|'.join(include_words), case=False)].drop_duplicates()
result['Kamick_PLS_LeadTimeDifference'] = result['Kamick Lead Time'] - result['PLS Lead Time']
result_WIRE['Kamick_PLS_LeadTimeDifference'] = result_WIRE['Kamick Lead Time'] - result_WIRE['PLS Lead Time']
# Sort for the top 50 lead time Differences in descending order and calculate essential statistics for both WIRE and NON-WIRE data
result50 = Result.sort_values(by='Kamick_PLS_LeadTimeDifference', ascending=False).head(50)
result50_wire = result_WIRE.sort_values(by='Kamick_PLS_LeadTimeDifference', ascending=False).head(50)
average_lead_time = result50['Kamick_PLS_LeadTimeDifference'].mean()
std_dev_lead_time = result50['Kamick_PLS_LeadTimeDifference'].std()
highest_lead_time = result50['Kamick_PLS_LeadTimeDifference'].max()
lowest_lead_time = result50['Kamick_PLS_LeadTimeDifference'].min()
average_lead_time_wire = result50_wire['Kamick_PLS_LeadTimeDifference'].mean()
std_dev_lead_time_wire = result50_wire['Kamick_PLS_LeadTimeDifference'].std()
highest_lead_time_wire = result50_wire['Kamick_PLS_LeadTimeDifference'].max()
lowest_lead_time_wire = result50_wire['Kamick_PLS_LeadTimeDifference'].min()
# Set up plotting style
sns.set(style="white")
logo_path1 = r"C:\Users\daddy\Downloads\NiSource Supply Chain Market Insights.pptx.png"
# Create PDF for storing plots and insights as needed
with PdfPages('lead_time_plots.pdf') as pdf:
    # Add cover page with our company logo
    fig = plt.figure(figsize=(8.27, 11.69))
    img = plt.imread(logo_path1)
    plt.imshow(img)
    plt.axis('off')
    pdf.savefig(fig, bbox_inches='tight')
    plt.close()
    # Create the Table of Contents (Page 2)
    plt.figure(figsize=(6, 8))
    toc_text = (
        "Table of Contents\n\n"
        "Page 2: Table Of Contents\n"
        "Page 3: AI Analysis for PLS and Kamick NON-WIRE data\n"
        "Page 4: AI Analysis for PLS and Kamick WIRE-ONLY data\n"
        "Page 5: Dataframe Visualization for NON-WIRE Kamick/PLS Data"
        "Page 6: DataFrame Visualization For WIRE ONLY Kamick and PLS Data\n"
    )
    plt.text(0, 1, toc_text, va='top', ha='left', fontsize=14, wrap=True)
    plt.axis('off')
    pdf.savefig()
    plt.close()
    #Generate AI insights for NON-WIRE data
    prompt1 = "Analyzing the statistics for the Difference In Lead Time Between the Kamick and PLS NON-WIRE data.\n\n"
    insight1 = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt1 + f"Average Lead Time Difference between Kamick and PLS: {average_lead_time}\n"
                        f"Standard Deviation Of The Lead Time Difference between Kamick and PLS: {std_dev_lead_time}\n"
                        f"Highest Lead Time Difference between Kamick and PLS: {highest_lead_time}\n"
                        f"Lowest Lead Time Difference between Kamick and PLS: {lowest_lead_time}\n",
        max_tokens=90,
        n=1
    )
    insight1_text = insight1.choices[0].text.strip()
    # Add AI insights for NON-WIRE data to the PDF
    plt.figure(figsize=(8, 6))
    plt.axis('off')
    plt.text(0.5, 0.5, insight1_text, ha='center', va='center', wrap=True)
    pdf.savefig()
    plt.close()
    #Generate AI insights for WIRE ONLY data
    prompt2 = "Analyzing the statistics for the Difference In Lead Time Between the Kamick and PLS WIRE ONLY data.\n\n"
    insight2 = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt2 + f"Average Lead Time Difference between Kamick and PLS: {average_lead_time_wire}\n"
                        f"Standard Deviation Of The Lead Time Difference between Kamick and PLS: {std_dev_lead_time_wire}\n"
                        f"Highest Lead Time Difference between Kamick and PLS: {highest_lead_time_wire}\n"
                        f"Lowest Lead Time Difference between Kamick and PLS: {lowest_lead_time_wire}\n",
        max_tokens=90,
        n=1
    )
    insight2_text = insight2.choices[0].text.strip()
    # Adding AI insights for WIRE ONLY data to the PDF
    plt.figure(figsize=(8, 6))
    plt.axis('off')
    plt.text(0.5, 0.5, insight2_text, ha='center', va='center', wrap=True)
    pdf.savefig()
    plt.close()
    # Create a new DataFrame containing only the columns the bosses want to see for NON-WIRE data
    table_data = result50[['Description', 'Kamick_PLS_LeadTimeDifference', 'Kamick Lead Time', 'PLS Lead Time']].copy()
    # Truncate/summarize the Description column to 40 characters
    table_data['Description'] = table_data['Description'].str[:40]
    # Convert 'Kamick_PLS_LeadTimeDifference' column to numeric data type
    table_data['Kamick_PLS_LeadTimeDifference'] = pd.to_numeric(table_data['Kamick_PLS_LeadTimeDifference'], errors='coerce')
    # ENSURE 'Kamick_PLS_LeadTimeDifference' is in descending order
    table_data = table_data.sort_values(by='Kamick_PLS_LeadTimeDifference', ascending=False)
    # Add top_fifty DataFrame for NON-WIRE data as a new page (Page 8)
    plt.figure(figsize=(8, 12))
    cell_height = 1 
    cell_colours = [['#C0C0C0'] * len(table_data.columns)] * (len(table_data) + 1)
    col_headers = table_data.columns.tolist()
    # Insert the column headers as the first row of table_data or the NON-WIRE data
    table_data = pd.concat([pd.DataFrame([col_headers], columns=table_data.columns), table_data], ignore_index=True)
    table = plt.table(cellText=table_data.values,
                      colLabels=None,
                      cellLoc='center', loc='center', cellColours=cell_colours)
    table.auto_set_font_size(False)
    table.set_fontsize(8)
    table.scale(1, cell_height)
    plt.axis('off')
    pdf.savefig()
    plt.close()
    # Create a new DataFrame containing only the columns the bosses want to see for WIRE only data
    Table_data = result50_wire[['Description', 'Kamick_PLS_LeadTimeDifference', 'Kamick Lead Time', 'PLS Lead Time']].copy()
    # Truncate and or summarize the Description column to 40 characters
    Table_data['Description'] = Table_data['Description'].str[:40]
    # Convert 'Kamick_PLS_LeadTimeDifference' column to numeric data type again
    Table_data['Kamick_PLS_LeadTimeDifference'] = pd.to_numeric(Table_data['Kamick_PLS_LeadTimeDifference'], errors='coerce')
    # Ensure Table_data by 'Kamick_PLS_LeadTimeDifference' is in descending order
    Table_data = Table_data.sort_values(by='Kamick_PLS_LeadTimeDifference', ascending=False)
    # Add top_fifty DataFrame for WIRE-ONLY data as a new page (Page 9)
    plt.figure(figsize=(8, 12))
    cell_height = 1
    Cell_colours = [['#C0C0C0'] * len(Table_data.columns)] * (len(Table_data) + 1)
    Col_headers = Table_data.columns.tolist()
    # Insert the column headers as the first row of Table_data or the WIRE ONLY data
    Table_data = pd.concat([pd.DataFrame([Col_headers], columns=Table_data.columns), Table_data], ignore_index=True)
    table = plt.table(cellText=Table_data.values,
                      colLabels=None,
                      cellLoc='center', loc='center', cellColours=Cell_colours)
    table.auto_set_font_size(False)
    table.set_fontsize(8)
    table.scale(1, cell_height)
    plt.axis('off')
    pdf.savefig()
    plt.close()
print("Plots and insights saved to lead_time_plots.pdf")

Plots and insights saved to lead_time_plots.pdf


In [ ]:
Result = Result.to_csv('Non-wire_Kamick_PLS_Data.csv')

In [ ]:
result_WIRE = result_WIRE.to_csv('Wire-ONLY_Kamick_PLS_Data.csv')